In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import sys

sys.path.append("..")
from plstm.longrange_transition_layer import (
    LongRangeTransitionLayer,
    LongRangeTransitionLayerConfig,
)

In [ ]:
D = 8
H = 4
B = 3

lrt = LongRangeTransitionLayer(
    LongRangeTransitionLayerConfig(
        input_dim=D,
        transition_dim=D,
        num_heads=H,
        normalization_mode="eigenvalue_restriction",
        inproj_weight_init="normal",
        outproj_weight_init="normal",
    )
)

In [ ]:
x = 0.1 * torch.randn([B, B, H, D])
y = lrt(x)

In [ ]:
y.shape

In [ ]:
svd = torch.linalg.svd(y[0, 0, 0])

In [ ]:
y[0, 0, 0].round(decimals=2)

In [ ]:
(
    torch.linalg.eigvals(y[0, 0, 0]).abs().max(),
    torch.linalg.eigvals(y[0, 0, 0]).abs().min(),
)

In [ ]:
from plstm.source_mark_layer import (
    SourceLayer,
    SourceLayerConfig,
    MarkLayer,
    MarkLayerConfig,
    DirectLayer,
    DirectLayerConfig,
)

In [ ]:
s = SourceLayer(
    SourceLayerConfig(
        JK=1,
        JV=1,
        JT=1,
        input_dim=D,
        num_heads=H,
    )
)

In [ ]:
s(x).shape

In [ ]:
m = MarkLayer(
    MarkLayerConfig(
        JO=1,
        JQ=1,
        JT=1,
        input_dim=D,
        num_heads=H,
    )
)

In [ ]:
m(x).shape

In [ ]:
d = DirectLayer(
    DirectLayerConfig(
        JO=2,
        JQ=1,
        JK=1,
        JV=2,
        input_dim=D,
        num_heads=H,
    )
)

In [ ]:
d(x)[0]

# Test orthogonalization strategies

### Exponentiation

In [ ]:
sys.path.append("..")
from plstm.longrange_transition_layer import matrix_orthogonalize_exponential

B, X = 1, 16
mat = torch.randn([B, X, X])  # torch.eye(2)[None, :, :] # torch.ones(1, 2, 2).tril()

mat.requires_grad_(True)

mat2 = mat.detach().clone()
mat2.requires_grad_(True)
mat1 = mat.detach().clone()
mat1.requires_grad_(True)

r1 = matrix_orthogonalize_exponential(mat1)
# r2 = matrix_orthogonalize_exponential(mat2, use_autograd=True)

print(r1)

In [ ]:
torch.linalg.eigvals(matrix_orthogonalize_exponential(mat)).abs()

In [ ]:
# matrix_orthogonalize_exponential(mat), matrix_orthogonalize_exponential(mat, use_autograd=True)

In [ ]:
# m = torch.randn_like(r1)
# m1 = m.clone().detach()
# m2 = m.clone().detach()

# l1 = (r1 * m1).sum().backward()
# l2 = (r2 * m2).sum().backward()

# mat1.grad, mat2.grad